<h2>Assignment 4 - Multilayer Perceptron to Recognize Handwritten Digits [12 Marks]</h2>
<br>
<b>Due Date:</b> December 03 at 23.59.<br>


<h3>Multilayer Perceptron [9 Marks]</h3>

We will start by implementing the multilayer perceptron.

1. [0 Marks] Read the code that is provided to you. You should start with the constructor, where the weights of the model are initialized. The constructor `__init__` is already fully implemented for you. Please read carefully the instructions using the dictionary `self.weights`. You will implement something similar with the dictionary `self.cache`. Then, read the method `train`, which is also fully implemented for you. It is in this method that the three methods you will need to implement are invoked. The method `train` first calls the forward propagation step (method `forward`), where the outputs of the model are computed. Then it calls the backward step (method `backward`), where the gradients of all weights are computed. Finally, it calls the update step (`update`), where we use the gradients to update the weights of the model.
2. [2.5 Marks] Implement the method `forward`. In your implementation you should store in `self.cache` the `Zi` and `Ai` values computed in the forward pass (see the lecture notes for details).
3. [4 Marks] Implement the `backward` method. In your implementation you should store in `self.cache` the `dW` and `dB` values with the gradients of all weights of the model. <b>You should assume the Cross Entropy loss</b>. This means that we don't multiply the pure error in the output layer by the derivative of the logistic function while computing $\Delta^{(L-1)}$. We still need to use the derivative of the logistic function when computing the $\Delta^{(i)}$ for the units in the hidden layers.
4. [2.5 Marks] Implement the `update` method. In this method you will use the values of `dW` and `dB` to update the weights of the model.


In [1]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np


class MLP:
    """
    Class implementing Multilayer Perceptron with sigmoid activation functions, which are used
    both in the hidden layers and in the output layer of the model.
    """

    def __init__(self, dims):
        """
        When creating an MLP object we define the architecture of the model (number of layers and number
        of units in each layer). The architecture is defined by a vector with a number of entries matching
        the number of units in each layer.

        For example, if dims = [784, 50, 20, 10], then the model has two hidden layers, with 50 and 20 neurons each,
        and an output layer with 10 units. The input layer receives 784 values.

        Since we are performing classification of handwritten digits, the input layer (784 units) and the output
        layer (10 units) are fixed: the input layer has one value for each pixel in the image, while the output
        layer has 10 units for each digit (0, 1, 2, ..., 9).

        The data of the model will be stored in dictionaries where the key is given by the name of the value store.
        In particular, we will have one dictionary called self.weights to store the weights and biases of the model;
        we will also have a dictionary called self.cache where we will store the matrices Z, A, and Delta matrices
        used in the backpropagation algorithm.

        The constructor also initializes the set of weights the model uses. For example, if dims = [784, 50, 20, 10],
        then the model has 3 sets of weights: one between the input layer and the first 50 units; another set of weights
        between 50 and 20; and a last set of weights between 20 and 10. We initialize the weights with small random numbers.
        The weights of the B vectors are initialized with zero.
        """
        self.dims = dims
        self.weights = {}
        self.L = len(dims)

        for i in range(1, len(dims)):
            self.weights[f"W{i-1}"] = np.random.randn(dims[i], dims[i - 1]) * (
                2 / np.sqrt(dims[i - 1])
            )
            self.weights[f"B{i-1}"] = np.zeros((dims[i], 1))

    def derivative(self, A):
        """
        Derivative of the logistic function
        """
        return np.multiply(A, 1 - A)

    def activation_function(self, Z):
        """
        Logistic function
        """
        return 1 / (1 + np.exp(-Z))

    def forward(self, X):
        """
        Forward pass. We initialize the self.cache dictionary with the vector representing the input layers, denoted A0.
        The forward pass then computes Zi and Ai until reaching the output layer. The last matrix A is then returned.
        """
        self.cache = {}
        self.cache["A0"] = X

        for i in range(1, self.L):
            self.cache[f"Z{i}"] = (
                np.dot(self.weights[f"W{i - 1}"], self.cache[f"A{i - 1}"])
                + self.weights[f"B{i - 1}"]
            )
            self.cache[f"A{i}"] = self.activation_function(self.cache[f"Z{i}"])

        return self.cache[f"A{self.L - 1}"]

    def backward(self, Y):
        """
        This function implements the backward step of the Backprop algorithm.

        The deltas di and gradients dW and dB are stored in self.cache with the keys di, dWi, and dBi, respectively.
        """

        self.cache[f"d{self.L - 1}"] = self.cache[f"A{self.L - 1}"] - Y

        for i in reversed(range(0, self.L - 1)):
            self.cache[f"dW{i}"] = np.dot(
                self.cache[f"d{i + 1}"], self.cache[f"A{i}"].T
            )
            self.cache[f"dB{i}"] = self.cache[f"d{i + 1}"]

            if i > 0:
                self.cache[f"d{i}"] = np.dot(
                    self.weights[f"W{i}"].T, self.cache[f"d{i + 1}"]
                ) * self.derivative(self.cache[f"A{i}"])

    def update(self, alpha):
        """
        Function must be called after backward is invoked.

        It will use the dWs and dBs stored in self.cache to update the weights self.weights of the model.
        """

        m = self.cache[f"A{self.L - 1}"].shape[1]

        for i in range(0, self.L - 1):
            self.weights[f"W{i}"] -= (alpha / m) * self.cache[f"dW{i}"]

            self.weights[f"B{i}"] -= ((alpha / m) * self.cache[f"dB{i}"]).sum(
                axis=1, keepdims=True
            )

    def train(self, X, Y, X_validation, Y_validation, alpha, steps):
        # creating one-hot encoding for the labels of the images
        Y_one_hot = np.zeros((10, X.shape[1]))
        for index, value in enumerate(Y):
            Y_one_hot[value][index] = 1

        # performs a number of gradient descent steps
        for i in range(0, steps):
            # computes matrices A and store them in self.cache
            self.forward(X)

            # computes matrices dW and dB and store them in self.cache
            self.backward(Y_one_hot)

            # use the matrices dW and dB to update the weights W and B of the model
            self.update(alpha)

            # every 100 training steps we print the accuracies of the model on a set of training and validation data sets
            if i % 100 == 0:
                percentage_train = self.evaluate(X, Y)
                percentage_validation = self.evaluate(X_validation, Y_validation)

                print(
                    "Accuracy training set %.3f, Accuracy validation set %.3f "
                    % (percentage_train, percentage_validation)
                )

    def evaluate(self, X, Y):
        """
        Receives a set of images stacked as column vectors in matrix X, their one-hot labels Y.

        Returns the percentage of images that were correctly classified by the model.
        """
        Y_hat = self.forward(X)
        classified_correctly = test_correct = np.count_nonzero(
            np.argmax(Y_hat, axis=0) == Y
        )
        return classified_correctly / X.shape[1]


Use the following code to test your implementation. We are following the scheme of stacking up input images as column vectors of `X`, as we did in Assignment 3 (see the lecture notes for Backpropagation for details).

We will run backpropagation for $1000$ training iterations on a data set with $20000$ training images. During training we will also compute the accuracy of the model on a set of $10000$, which we call the validation set.

<b>Disclaimer:</b> The outputs of the next cell won't be meaningful before you finish implementing the multilayer perceptron.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

m = 20000
val = 10000
images, labels = (x_train[0:m].reshape(m, 28 * 28) / 255, y_train[0:m])
images = images.T

images_validation, labels_validation = (
    x_train[m : m + val].reshape(val, 28 * 28) / 255,
    y_train[m : m + val],
)
images_validation = images_validation.T

dims = [784, 50, 20, 10]
mlp = MLP(dims)
mlp.train(images, labels, images_validation, labels_validation, 0.5, 3000)


Accuracy training set 0.106, Accuracy validation set 0.111 
Accuracy training set 0.775, Accuracy validation set 0.766 
Accuracy training set 0.855, Accuracy validation set 0.851 
Accuracy training set 0.888, Accuracy validation set 0.882 
Accuracy training set 0.904, Accuracy validation set 0.895 
Accuracy training set 0.913, Accuracy validation set 0.904 
Accuracy training set 0.921, Accuracy validation set 0.910 
Accuracy training set 0.928, Accuracy validation set 0.917 
Accuracy training set 0.934, Accuracy validation set 0.920 
Accuracy training set 0.938, Accuracy validation set 0.924 
Accuracy training set 0.942, Accuracy validation set 0.927 
Accuracy training set 0.946, Accuracy validation set 0.930 
Accuracy training set 0.949, Accuracy validation set 0.933 
Accuracy training set 0.952, Accuracy validation set 0.935 
Accuracy training set 0.955, Accuracy validation set 0.937 
Accuracy training set 0.957, Accuracy validation set 0.938 
Accuracy training set 0.959, Accuracy va

<h4>Question 1 [3 Marks]</h4>

How do the results you obtained with the multilayer perceptron compare with the results you obtained with Logistic Regression using Cross Entropy loss in Assignment 3? How do you justify the difference in accuracy observed in the two experiments?

The MLP model started slower than the Logistic Regression model with Cross Entropy loss however the accuracy of the MLP is higher at the end of the training. This is because the multilayer percepteron has a hidden layer that is better able to discover indirect corellations between the input and output values but since there are more values to train it take longer to ramp up.
